In [1]:
# Cálcula el porcentaje de error entre dos series de máximos anuales.

import itertools
import os

import numpy as np
import pandas as pd
import xarray as xr

from matplotlib import pyplot as plt

In [2]:
fdir = ["../"]
name = "CHIRPS_megalopolis"

fdir = [ "../data/" + name + ".nc", 
    "../results/" + name + "/" + name + "_tretorno.nc"  ]

ds_tot = xr.open_dataset( fdir[0] )
ds_tot = ds_tot.rename_vars( { "longitude": "LONGITUD", "latitude": "LATITUD",
    "time": "TIEMPO", "precip": "PRECIPITACION" } ).swap_dims(
    { "longitude": "LONGITUD", "latitude": "LATITUD", "time": "TIEMPO" } 
    ).to_dataframe().to_xarray()
ds_trt = xr.open_dataset( fdir[1] ).drop( ["AÑO", "PROBABILIDAD"] )

ext = ds_tot["PRECIPITACION"].shape[1:3]

ds_tot

<xarray.Dataset>
Dimensions:        (TIEMPO: 15340, LONGITUD: 64, LATITUD: 52)
Coordinates:
  * TIEMPO         (TIEMPO) datetime64[ns] 1981-01-01 1981-01-02 ... 2022-12-31
  * LONGITUD       (LONGITUD) float64 -100.5 -100.4 -100.4 ... -97.38 -97.33
  * LATITUD        (LATITUD) float64 18.12 18.17 18.22 ... 20.57 20.62 20.67
Data variables:
    PRECIPITACION  (TIEMPO, LONGITUD, LATITUD) float32 0.0 0.0 0.0 ... 0.0 0.0

In [3]:
df_i = ds_tot.isel( { "LONGITUD": slice(0, ext[0]),
    "LATITUD": slice(0, ext[1]) } ).mean(
    ["LONGITUD", "LATITUD"] ).to_dataframe()
df_i["CURVA_MASA"] = df_i["PRECIPITACION"].cumsum()

df_trt = ds_trt.sel( {"DURACION": 1} ).drop( "DURACION"
    ).isel( { "LONGITUD": slice(0, ext[0]),
    "LATITUD": slice(0, ext[1]) } ).mean(
    ["LONGITUD", "LATITUD"] ).to_dataframe()

df_i

,PRECIPITACION,CURVA_MASA
TIEMPO,,
1981-01-01,0.003477,0.003477
1981-01-02,0.000480,0.003957
1981-01-03,0.000000,0.003957
1981-01-04,0.000000,0.003957
1981-01-05,0.011128,0.015085
...,...,...
2022-12-27,0.005312,38397.503906
2022-12-28,0.000000,38397.503906
2022-12-29,0.369039,38397.871094


In [4]:
dt = 1

df_sam = ( df_i[ ["PRECIPITACION"] ] / dt ).rename(
    {"PRECIPITACION": "INTENSIDAD"}, axis = 1 
    ).groupby( df_i.index.year ).max().sort_values(
    "INTENSIDAD", ascending = False ).reset_index(
    drop = True ).rename( {"TIEMPO": "AÑO"}, axis = 1 )
df_sam.index.name = "T_RETORNO"
df_sam.index += 1
# Calculamos el tiempo de retorno.
df_sam.index = (df_sam.index[-1] + 1) / df_sam.index
df_sam = df_sam.sort_index()

df_sam

,INTENSIDAD
T_RETORNO,
1.023810,15.664720
1.048780,16.127346
1.075000,17.973457
1.102564,18.776258
1.131579,19.379120
1.162162,19.387455
1.194444,19.891775
1.228571,20.090275
1.264706,20.283638


In [5]:
A = ext[0] * 2 * ext[1] * 2
A

13312

In [6]:
df_arf = df_sam / df_trt
df_arf.head()

,INTENSIDAD
T_RETORNO,
1.023810,0.605469
1.048780,0.567964
1.075000,0.599053
1.102564,0.600459
1.131579,0.598898


In [7]:
df_sam = df_i.copy()
for i in ds_trt["DURACION"].values:
    df_sam[ dt * i ] = ( ( df_sam["CURVA_MASA"]
        - df_sam["CURVA_MASA"].shift(i) ) #/ i
        )
df_sam.columns.name = "DURACION"
df_sam = df_sam.drop( ["PRECIPITACION", "CURVA_MASA"],
    axis = 1 ).groupby( df_sam.index.year ).max()
df_sam = df_sam.apply( lambda x: x.sort_values(
    ascending = False ).values, axis = 0 )
# Calculamos el tiempo de retorno.
df_sam = df_sam.reset_index( drop = True )
df_sam.index.name = "T_RETORNO"
df_sam.index += 1
df_sam.index = (df_sam.index[-1] + 1) / df_sam.index
df_sam = df_sam.sort_index()

df_sam

DURACION,1,2,3,4,5,10,15,20,30,40,50,60
T_RETORNO,,,,,,,,,,,,
1.023810,15.664551,24.535156,34.319824,42.754883,50.440430,78.720703,85.897949,96.719727,150.976807,183.123169,217.110107,254.662720
1.048780,16.127319,24.840332,34.335938,43.672852,50.751953,79.509766,99.218750,123.887695,180.028320,218.444336,262.822266,300.035156
1.075000,17.973633,27.271484,37.269531,45.777832,51.307129,80.481567,104.340820,134.351562,187.902344,227.277344,266.468750,304.862305
1.102564,18.777344,27.824219,38.066406,46.163086,51.491211,83.832031,106.376953,137.238281,190.146973,238.574219,281.138672,326.849609
1.131579,19.378906,29.052734,38.507812,46.181030,52.091797,84.693359,106.613281,142.211426,194.308594,240.902344,290.695312,335.367188
1.162162,19.386719,29.091797,39.624023,46.441406,52.931641,85.064453,117.015625,145.938477,195.658203,244.567383,293.960938,337.220703
1.194444,19.892578,30.585938,39.789062,47.203125,53.281250,85.343750,117.433594,148.948242,196.236328,244.608398,296.316406,344.466797
1.228571,20.089844,30.949951,40.691406,47.632812,55.754517,86.007812,119.768066,149.496094,201.763672,244.681641,297.483887,348.522461
1.264706,20.283203,32.041016,40.859375,48.777344,57.144531,86.525391,120.195312,149.735352,202.073242,245.670410,298.724609,348.841797


In [8]:
df_trt = ds_trt.isel( { "LONGITUD": slice(0, ext[0]),
    "LATITUD": slice(0, ext[1]) } ).mean(
    ["LONGITUD", "LATITUD"] ).to_dataframe()
df_trt = df_trt.unstack().T
df_trt = df_trt.reset_index(level = 0, drop = True)
df_trt = df_trt.T.apply( lambda x: x * df_trt.T.index ).T

df_trt

DURACION,1,2,3,4,5,10,15,20,30,40,50,60
TIEMPO_RETORNO,,,,,,,,,,,,
1.023810,25.872049,36.701195,44.681156,51.316746,57.655811,84.705620,106.437407,126.739607,166.662083,204.398270,239.133859,274.054871
1.048780,28.395020,39.921516,48.591871,55.628845,62.393289,91.382332,115.175350,138.047123,181.722593,221.684475,259.174204,296.778917
1.075000,30.003094,41.952126,50.929115,58.190952,65.307531,95.490503,120.762606,145.321045,191.261129,233.428955,273.408127,311.619987
1.102564,31.269854,43.510788,52.851007,60.344387,67.570887,98.666697,125.064611,150.512018,198.184147,241.580181,282.705855,322.375059
1.131579,32.357964,44.885689,54.458593,62.234959,69.516788,101.414776,128.615427,154.882593,204.023209,248.139782,290.384626,331.017265
1.162162,33.292824,46.084942,55.869993,63.903259,71.304479,103.859768,131.824737,158.690643,208.987398,254.037609,296.930099,337.948952
1.194444,34.148857,47.212730,57.143703,65.460442,72.972760,106.176624,134.847136,162.266426,213.376236,258.993855,302.758694,344.093800
1.228571,34.966976,48.261467,58.364679,66.883911,74.513173,108.318176,137.695141,165.742569,217.622466,263.586769,307.849860,349.600325
1.264706,35.788345,49.259418,59.544153,68.232979,76.012578,110.355272,140.263395,168.849506,221.371222,268.082867,312.858081,354.941368


In [9]:
df_arf = df_sam / df_trt
df_arf

DURACION,1,2,3,4,5,10,15,20,30,40,50,60
T_RETORNO,,,,,,,,,,,,
1.023810,0.605462,0.668511,0.768105,0.833157,0.874854,0.929345,0.807028,0.763137,0.905886,0.895913,0.907902,0.929240
1.048780,0.567963,0.622229,0.706619,0.785076,0.813420,0.870078,0.861458,0.897430,0.990677,0.985384,1.014076,1.010972
1.075000,0.599059,0.650062,0.731792,0.786683,0.785623,0.842823,0.864016,0.924516,0.982439,0.973647,0.974619,0.978314
1.102564,0.600493,0.639479,0.720259,0.764994,0.762032,0.849649,0.850576,0.911809,0.959446,0.987557,0.994456,1.013880
1.131579,0.598891,0.647261,0.707103,0.742043,0.749341,0.835119,0.828931,0.918189,0.952385,0.970833,1.001070,1.013141
1.162162,0.582309,0.631265,0.709218,0.726746,0.742333,0.819032,0.887661,0.919641,0.936220,0.962721,0.990000,0.997845
1.194444,0.582525,0.647832,0.696298,0.721094,0.730153,0.803790,0.870865,0.917924,0.919673,0.944456,0.978721,1.001084
1.228571,0.574538,0.641297,0.697192,0.712171,0.748250,0.794029,0.869806,0.901978,0.927127,0.928277,0.966328,0.996917
1.264706,0.566754,0.650455,0.686203,0.714865,0.751777,0.784062,0.856926,0.886798,0.912825,0.916397,0.954825,0.982815


In [10]:
pre = "PRECIPITACION"
a = ds_tot.rolling({"TIEMPO": 60}).sum().sel(TIEMPO = "2000-07-19"
    ).mean( ["LONGITUD", "LATITUD"] )[pre].values
b = ds_tot.rolling({"TIEMPO": 60}).sum().max(dim = "TIEMPO"
    ).mean(["LONGITUD", "LATITUD"])[pre].values
a, b

NameError: name 'pre' is not defined

In [ ]:
# Cálculo de SAM.

pre = "PRECIPITACION"
dur = [1, 2,  3,  4,  5, 10, 15, 20, 30, 40, 50, 60]

j = 0
ds_i = ds_tot.isel( { "LONGITUD": slice( j, ext[0] - j ),
    "LATITUD": slice( 0, ext[1] - 0 ) } )

years = ds_i["TIEMPO"].dt.year.to_dataframe()["year"].unique()
ny = years.shape[0]

ds_tret = []

for i in range( len(dur) ):
    ds_tret.append( ds_i.isel( {"TIEMPO": slice(0, ny)} ).copy() )
    ds_tret[i]["TIEMPO"] = years
    ds_tret[i]["AÑO"] = ds_tret[i]["PRECIPITACION"].copy()
    ds_p = ds_tret[i]["PRECIPITACION"]
    ds_a = ds_tret[i]["AÑO"]

    t = 60
    ds_t = ds_i.rolling( {"TIEMPO": dur[i]} ).sum()
    for y in years:
        ds_p[ ds_p["TIEMPO"] == y ] = ( ds_t[pre].sel(
            { "TIEMPO": (ds_t["TIEMPO"].dt.year == y) }
            ).max(dim = "TIEMPO") )

    for lon in ds_tret[i]["LONGITUD"]:
        for lat in ds_tret[i]["LATITUD"]:
            df_p = ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat
                ].to_dataframe().sort_values( pre, ascending = True 
                ).reset_index().rename( {"TIEMPO": "AÑO"}, axis = 1 )
            df_p["TIEMPO"] = years
            df_p = df_p.set_index(["TIEMPO", "LONGITUD", "LATITUD"])
            ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat ] = (
                df_p.to_xarray()[pre] )
            ds_a[ :, ds_a["LONGITUD"] == lon, ds_a["LATITUD"] == lat ] = (
                df_p.to_xarray()["AÑO"] )
        
    ds_tret[i]["TIEMPO"] = ( ny + 1 ) / np.arange(ny, 0, -1)
    ds_tret[i] = ds_tret[i].rename_dims( {"TIEMPO": "TIEMPO_RETORNO"} 
                          ).rename_vars( {"TIEMPO": "TIEMPO_RETORNO"} )
    
    ds_tret[i] = ds_tret[i].expand_dims( {"DURACION": [dur[i]] } )

ds_trt = xr.concat(ds_tret, "DURACION")

ds_trt

<xarray.Dataset>
Dimensions:         (TIEMPO_RETORNO: 42, LONGITUD: 64, LATITUD: 52, DURACION: 1)
Coordinates:
  * DURACION        (DURACION) int64 60
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 1.024 1.049 1.075 ... 21.5 43.0
  * LONGITUD        (LONGITUD) float64 -100.5 -100.4 -100.4 ... -97.38 -97.33
  * LATITUD         (LATITUD) float64 18.12 18.17 18.22 ... 20.57 20.62 20.67
Data variables:
    PRECIPITACION   (DURACION, TIEMPO_RETORNO, LONGITUD, LATITUD) float32 350...
    AÑO             (DURACION, TIEMPO_RETORNO, LONGITUD, LATITUD) float32 2.0...

In [41]:
# Cálculo de ARF.

pre = "PRECIPITACION"
dur = [1, 2,  3,  4,  5, 10, 15, 20, 30, 40, 50, 60]

ds_trt = []

j_a = min( ds_tot["LONGITUD"].shape[0] / 2 - 1,
           ds_tot["LATITUD" ].shape[0] / 2 - 1 )

for j in range(0, j_a):
    ds_i = ds_tot.isel( { "LONGITUD": slice( j, ext[0] - j ),
        "LATITUD": slice( 0, ext[1] - 0 ) } )

    years = ds_i["TIEMPO"].dt.year.to_dataframe()["year"].unique()
    ny = years.shape[0]

    ds_tret = []

    for i in range( len(dur) ):
        ds_tret.append( ds_i.isel( {"TIEMPO": slice(0, ny)} ).copy() )
        ds_tret[i]["TIEMPO"] = years
        ds_p = ds_tret[i]["PRECIPITACION"]

        t = 60
        ds_t = ds_i.rolling( {"TIEMPO": dur[i]} ).sum()
        for y in years:
            ds_p[ ds_p["TIEMPO"] == y ] = ( ds_t[pre].sel(
                { "TIEMPO": (ds_t["TIEMPO"].dt.year == y) }
                ).max(dim = "TIEMPO") )

        for lon in ds_tret[i]["LONGITUD"]:
            for lat in ds_tret[i]["LATITUD"]:
                df_p = ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat
                    ].to_dataframe().sort_values( pre, ascending = True 
                    ).reset_index().rename( {"TIEMPO": "AÑO"}, axis = 1 )
                df_p["TIEMPO"] = years
                df_p = df_p.set_index(["TIEMPO", "LONGITUD", "LATITUD"])
                ds_p[ :, ds_p["LONGITUD"] == lon, ds_p["LATITUD"] == lat ] = (
                    df_p.to_xarray()[pre] )
        
        ds_tret[i]["TIEMPO"] = ( ny + 1 ) / np.arange(ny, 0, -1)
        ds_tret[i] = ds_tret[i].rename_dims( {"TIEMPO": "TIEMPO_RETORNO"} 
                              ).rename_vars( {"TIEMPO": "TIEMPO_RETORNO"} )
    
        ds_tret[i] = ds_tret[i].mean( ["LONGITUD", "LATITUD"]
            ).expand_dims( {"DURACION": [dur[i]] } )

    ds_tret = xr.concat(ds_tret, "DURACION")


    df_i = ds_i.mean(
        ["LONGITUD", "LATITUD"] ).to_dataframe()
    df_i["CURVA_MASA"] = df_i["PRECIPITACION"].cumsum()

    #df_sam = df_i.copy()
    for i in dur:
        df_i[ dt * i ] = ( ( df_i["CURVA_MASA"]
            - df_i["CURVA_MASA"].shift(i) ) )
    df_i.columns.name = "DURACION"
    df_i = df_i.drop( ["PRECIPITACION", "CURVA_MASA"],
        axis = 1 ).groupby( df_i.index.year ).max()
    df_i = df_i.apply( lambda x: x.sort_values(
        ascending = False ).values, axis = 0 )
    
    # Calculamos el tiempo de retorno.
    df_i = df_i.reset_index( drop = True )
    df_i.index.name = "T_RETORNO"
    df_i.index += 1
    df_i.index = ( df_i.index[-1] + 1 ) / df_i.index
    df_i = df_i.sort_index()

    ds_tret = df_i.stack().to_xarray() / ds_tret

    ds_trt.append( ds_tret.expand_dims( { "AREA": [ (
        ds_i["LONGITUD"].shape[0] * 5 * ds_i["LONGITUD"].shape[0] * 5 ) ] } ) )

ds_trt = xr.concat(ds_trt, "AREA")

ds_trt

<xarray.Dataset>
Dimensions:         (AREA: 1, DURACION: 12, TIEMPO_RETORNO: 42, T_RETORNO: 42)
Coordinates:
  * AREA            (AREA) int64 102400
  * DURACION        (DURACION) int64 1 2 3 4 5 10 15 20 30 40 50 60
  * TIEMPO_RETORNO  (TIEMPO_RETORNO) float64 1.024 1.049 1.075 ... 21.5 43.0
  * T_RETORNO       (T_RETORNO) float64 1.024 1.049 1.075 ... 14.33 21.5 43.0
Data variables:
    PRECIPITACION   (AREA, T_RETORNO, DURACION, TIEMPO_RETORNO) float32 0.605...

In [6]:
df_sam = df_i.copy()
for i in ds_trt["DURACION"].values:
    df_sam[ dt * i ] = ( ( df_sam["CURVA_MASA"]
        - df_sam["CURVA_MASA"].shift(i) ) #/ i
        )
df_sam.columns.name = "DURACION"
df_sam = df_sam.drop( ["PRECIPITACION", "CURVA_MASA"],
    axis = 1 ).groupby( df_sam.index.year ).max()
#df_sam = df_sam.apply( lambda x: x.sort_values(
#    ascending = False ).values, axis = 0 )
df_sam.sort_values(1)

DURACION,1,2,3,4,5,10,15,20,30,40,50,60
TIEMPO,,,,,,,,,,,,
1986,15.664551,24.840332,34.335938,42.754883,52.931641,88.182617,128.693848,158.888184,224.983398,268.941406,325.329590,356.100098
1982,16.127319,24.535156,34.319824,46.181030,55.754517,80.481567,85.897949,96.719727,150.976807,183.123169,217.110107,254.662720
1987,17.973633,29.052734,39.624023,45.777832,51.307129,92.020508,123.743164,159.209961,202.576660,266.167480,313.961426,361.898438
2019,18.777344,27.824219,38.507812,50.027344,60.652344,91.457031,106.613281,134.351562,194.308594,238.574219,293.960938,355.425781
1989,19.378906,33.462402,44.028320,50.866699,57.329590,95.709473,119.768066,145.938477,190.146973,245.670410,297.483887,348.522461
2001,19.386719,35.013672,40.691406,46.441406,52.091797,86.007812,121.755859,156.667969,217.423828,244.681641,296.316406,348.841797
2015,19.892578,33.679688,43.759766,50.644531,57.144531,78.720703,106.376953,149.496094,195.658203,227.277344,262.822266,300.035156
2010,20.089844,36.478516,54.167969,66.859375,76.664062,127.777344,155.556641,192.759766,285.722656,329.943359,412.285156,461.988281
2009,20.283203,34.873047,48.564453,60.474609,71.332031,135.144531,172.703125,212.933594,270.880859,313.529297,342.300781,378.912109
